In [7]:
!pip install sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.kl import KLSummarizer
import pandas as pd, nltk, re
from nltk.tokenize import sent_tokenize, word_tokenize
pd.options.mode.chained_assignment = None  # default='warn'

In [8]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_train.csv', encoding='utf-8')

In [9]:
stopword_list = nltk.corpus.stopwords.words('german')
LANGUAGE = "german"
def remove_stopwords(text):
    filtered_words = [word for word in nltk.word_tokenize(text, language="german") if word not in stopword_list]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def KLdiv(df, size):
    lst = []
    parser = PlaintextParser.from_string(df, Tokenizer(LANGUAGE))
    summarizer = KLSummarizer()
    summary = summarizer(parser.document, size)
    for s in summarizer(parser.document, size):
        lst.append(s)
    string = ' '.join([str(e) for e in lst])
    return string

In [10]:
test_df = df.head(50)
print(KLdiv(test_df['source'][0], 1))

Gegen Ende seines Lebens kehrte Minghella zu seinen Anfängen im Radio und auf der Bühne zurück: 2006 wurde sein Hörspiel "Eyes Down Looking" mit Jude Law zu Ehren von Samuel Beckett auf BBC Radio 3 ausgestrahlt, ein Jahr zuvor hatte seine Inszenierung der Puccini-Oper Madame Butterfly in der English National Opera in London Premiere und wurde auch in der Nationaloper von Vilnius und in der Metropolitan Opera in New York gezeigt.


In [12]:
test_df['source'] = test_df['source'].apply(remove_stopwords)
test_df['kl1'] = test_df.apply(lambda x: KLdiv(x['source'], 1), axis=1)
test_df['kl2'] = test_df.apply(lambda x: KLdiv(x['source'], 2), axis=1)
test_df['kl5'] = test_df.apply(lambda x: KLdiv(x['source'], 5), axis=1)
test_df['kl10'] = test_df.apply(lambda x: KLdiv(x['source'], 10), axis=1)
test_df.to_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/kl-divergenceR2.csv')

In [ ]:
from rouge import Rouge
import sys

sys.stdout = open("G:/Extractive-Summarisation-of-German-Wikipedia/dataset/kl_test.txt", "w")

hy, rf = test_df['kl1'], test_df['summary']
rouge= Rouge()
scoresT1= rouge.get_scores(hy, rf, avg=True)
print("KL1 scores: ", scoresT1, '\n\n')

hy2 = test_df['kl2']
scoresT2= rouge.get_scores(hy2, rf, avg=True)
print("KL2 scores: ", scoresT2, '\n\n')

hy5 = test_df['kl5']
scoresT5= rouge.get_scores(hy5, rf, avg=True)
print("KL5 scores: ", scoresT5, '\n\n')

hy10 = test_df['kl10']
scoresT10= rouge.get_scores(hy10, rf, avg=True)
print("KL10 scores: ", scoresT10, '\n\n')

sys.stdout.close()